## Import libraries

In [1]:
import sys
sys.path.append("../")

# Basic libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Computer Vision 
import cv2
from PIL import Image
from skimage import measure
import datetime

#Deep learning libraries  
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from keras import objectives
from keras.utils import plot_model
from tensorflow.keras import optimizers


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Concatenate
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, UpSampling2D, Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import ELU
from tensorflow.keras.layers import GlobalAveragePooling2D

#Custom
import config
from src.models.callbacks import tensorboard,checkpoint,reduce_lr
from src.visualization.visualize import plot_learning_curve
from config import intermediate_dim,latent_dim,batch_size,epochs

Using TensorFlow backend.


# Load Data

In [2]:
X_train = np.load(os.path.join(config.DATA_PREPROCESSED_DIR,"X_train.npy"))
X_valid = np.load(os.path.join(config.DATA_PREPROCESSED_DIR,"X_valid.npy"))

y_train = np.load(os.path.join(config.DATA_PREPROCESSED_DIR,"y_train.npy"))
y_valid = np.load(os.path.join(config.DATA_PREPROCESSED_DIR,"y_valid.npy"))

## Model architecture

In [3]:
def build_vae(input_shape):

    #input_shape=(30,30,1)
    
    width = input_shape[0]
    height = input_shape[1]
    channels = input_shape[2] 
    
    #encoder
    input_img = Input(batch_shape=(None,30,30,1))
    squeeze=Conv2D(256, 3,3,padding='same',input_shape=input_shape)(input_img)
    squeeze=Activation('relu')(squeeze)
    squeeze=BatchNormalization()(squeeze)
    squeeze=MaxPooling2D(pool_size=(2,2))(squeeze)
    squeeze=UpSampling2D(size=(2, 2))(squeeze)

    #bottleneck
    squeeze=Conv2D(128, 3,3, kernel_initializer='glorot_uniform')(squeeze)

    squeeze0=Activation('relu')(squeeze)
    squeeze0=Dense(100)(squeeze0)
    squeeze0=Reshape((30,30))(squeeze0)
    squeeze0=Activation('relu')(squeeze0)

    model = Model(inputs = input_img, outputs = squeeze0)
    
    return model    

In [4]:
input_shape=(30,30,1)
vae = build_vae(input_shape)
vae.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 30, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 10, 10, 256)       2560      
_________________________________________________________________
activation (Activation)      (None, 10, 10, 256)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 10, 256)       1024      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 5, 256)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 10, 10, 256)       0         
_________

# Reshape to model

In [9]:
n_train_samples = X_train.shape[0]
n_valid_samples = X_valid.shape[0]

In [16]:
X_train = X_train.reshape(n_train_samples,input_shape[1],input_shape[1],1)
y_train = y_train.reshape(n_train_samples,input_shape[1],input_shape[1])
X_valid = X_valid.reshape(n_valid_samples,input_shape[1],input_shape[1],1)
y_valid = y_valid.reshape(n_valid_samples,input_shape[1],input_shape[1])

In [17]:
print("Shapes before training\n")
print("X_train : ",X_train.shape)
print("y_train : ",y_train.shape)
print("X_valid : ",X_valid.shape)
print("y_valid : ",y_valid.shape)

Shapes before training

X_train :  (900, 30, 30, 1)
y_train :  (900, 30, 30)
X_valid :  (120, 30, 30, 1)
y_valid :  (120, 30, 30)


# Set optimizer and loss

In [7]:
optimizer = optimizers.RMSprop(lr=0.0005, rho=0.9, epsilon=None, decay=0.0)
loss = 'mean_squared_error'
vae.compile(loss=loss, optimizer=optimizer)

In [8]:
history = vae.fit(X_train, y_train,verbose=1,
        epochs=500,validation_data=(X_valid,y_valid),
        batch_size=128,callbacks=[tensorboard,checkpoint,reduce_lr])

Train on 900 samples, validate on 120 samples
Epoch 1/500
896/900 [============================>.] - ETA: 0s - loss: 0.0900
Epoch 00001: val_loss improved from inf to 0.09126, saving model to /home/michal/Desktop/ML_Projects/eeg2image/models/VAE.hdf5
900/900 [==============================] - 2s 2ms/sample - loss: 0.0899 - val_loss: 0.0913
Epoch 2/500
896/900 [============================>.] - ETA: 0s - loss: 0.0845
Epoch 00002: val_loss improved from 0.09126 to 0.08698, saving model to /home/michal/Desktop/ML_Projects/eeg2image/models/VAE.hdf5
900/900 [==============================] - 1s 1ms/sample - loss: 0.0845 - val_loss: 0.0870
Epoch 3/500
896/900 [============================>.] - ETA: 0s - loss: 0.0821
Epoch 00003: val_loss did not improve from 0.08698
900/900 [==============================] - 1s 1ms/sample - loss: 0.0821 - val_loss: 0.0883
Epoch 4/500
896/900 [============================>.] - ETA: 0s - loss: 0.0802
Epoch 00004: val_loss improved from 0.08698 to 0.08355, savi

KeyboardInterrupt: 

In [ ]:
def plot_learning_curve(history, figsize=(15, 5)):
    fig,ax = plt.subplots(1,1,figsize=figsize)
    # Plot training & validation loss values
    ax.plot(history.history['loss'])
    if "val_loss" in history.history:
            ax.plot(history.history['val_loss'])
            plt.legend(['Train', 'Validation'], loc='upper right')
    ax.set_title('Model loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    plt.grid()

    return fig

In [ ]:
_ = plot_learning_curve(history)

In [ ]:
fig=plt.figure(figsize=(14, 6))
#plt.suptitle("Results",size = 20)

columns = 6
rows = 2

for i in range(1, columns*rows +1):
    if i  <= columns:    
   
        fig.add_subplot(rows, columns, i)
        img = y_train[i].reshape(30,30)
        plt.imshow(img,cmap=plt.cm.binary)
        plt.title("Stimulus")
        
    else : 
        
        fig.add_subplot(rows, columns, i)
        x = X_train[i-columns].reshape(1,30,30,1)
        x_test_encoded = model.predict(x, batch_size=1).reshape(30,30)
        plt.imshow(x_test_encoded,cmap=plt.cm.binary)
        
        plt.title("From Brain ")
plt.show()

# Evaluation

In [25]:
from tensorflow.keras.models import load_model

In [26]:
model_path = os.path.join(config.MODEL_DIR,"VAE.hdf5")

In [27]:
model = load_model(model_path)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
# load data

In [28]:
X_test = np.load(os.path.join(config.DATA_PREPROCESSED_DIR,"X_test.npy"))
y_test = np.load(os.path.join(config.DATA_PREPROCESSED_DIR,"y_test.npy"))

In [29]:
# reshape data

In [32]:
n_test_samples = X_test.shape[0]

In [33]:
X_test = X_test.reshape(n_test_samples,input_shape[1],input_shape[1],1)
y_test = y_test.reshape(n_test_samples,input_shape[1],input_shape[1])

In [38]:
x = model.evaluate(X_test,y_test)

180/180 [==============================] - 0s 267us/sample - loss: 0.0666


In [39]:
x

0.06656740870740678